In [1]:
from trainer import LargeLanguageModelTrainer
from lightning_modules import SimilarityModule
from data_utils import PhraseDataset
from transformer_class import BaseTransformerModel
import torch
import pytorch_lightning as pl
import transformers

/opt/anaconda3/envs/transformers/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
pretrained_model = BaseTransformerModel(model_name="bert-base-uncased")
tokenizer = transformers.AutoTokenizer.from_pretrained("bert-base-uncased")
train = PhraseDataset("processed/train.csv", tokenizer)
val = PhraseDataset("processed/valid.csv", tokenizer)
optimizer = torch.optim.Adam(pretrained_model.parameters(), lr=1e-5)
loss = torch.nn.MSELoss()
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=2, factor=0.5)
callbacks = [
    pl.callbacks.ModelCheckpoint(monitor="val_loss", save_top_k=1),
    pl.callbacks.EarlyStopping(monitor="val_loss", patience=5),
    pl.callbacks.LearningRateMonitor(logging_interval="step")
]
model_hyperparams  = {
    "loss": loss,
    "optimizer": optimizer,
    "scheduler": None,
    "lr": 1e-5
}
trainer_hyperparams = {
    'device': "cpu",
    'batch_size': 32,
    'num_workers': 4,
}
logger = pl.loggers.TensorBoardLogger("tb_logs", name="similarity_adam_1e-5_32_LROnPlateau")
model = SimilarityModule(model=pretrained_model, hyperparameters=model_hyperparams)
trainer = LargeLanguageModelTrainer(model=model,
                                    hyperparameters=trainer_hyperparams,
                                    train_dataset=train,
                                    eval_dataset=val,
                                    callbacks=callbacks,
                                    logger=logger
                                )
trainer.train_similarity()


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/opt/ana

Sanity Checking: 0it [00:00, ?it/s]huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | 

/opt/anaconda3/envs/transformers/lib/python3.9/site-packages/pytorch_lightning/trainer/trainer.py:653: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")
